In [1]:
import requests as re
import os

from IPython.display import JSON  
import pandas as pd

In [2]:
api_key = os.environ["GOGGLE_API_KEY"]

In [3]:
url= "https://maps.googleapis.com/maps/api/place/nearbysearch/json?location=51.0478,-114.0593&radius=5000&keyword=food&types=restaurants&key={}".format(api_key)


In [4]:
res = re.request("GET", url)

In [5]:
result= res.json()
JSON(result)

<IPython.core.display.JSON object>

In [6]:
import sqlite3
from sqlite3 import Error


In [7]:
def create_connection(path):
    connection = None
    try:
        connection = sqlite3.connect(path)
        print("Connection to SQLite DB successful")
    except Error as e:
        print(f"The error '{e}' occurred")
    return connection

In [8]:
connection = create_connection("downtown_calgary_POI.sqlite")

Connection to SQLite DB successful


In [9]:
def execute_query(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        connection.commit()
        print("Query executed successfully")
    except Error as e:
        print(f"The error '{e}' occurred")

In [10]:
#open JSON data
df = pd.json_normalize(result, "results")
df.head(2)
#remove come columns
df1= df.drop(columns=["place_id","photos","types","icon","plus_code.compound_code","plus_code.global_code","icon_mask_base_uri", "icon_background_color", "reference", "scope"])
df1.head(2)

,business_status,name,rating,user_ratings_total,vicinity,geometry.location.lat,geometry.location.lng,geometry.viewport.northeast.lat,geometry.viewport.northeast.lng,geometry.viewport.southwest.lat,geometry.viewport.southwest.lng,opening_hours.open_now,permanently_closed,price_level
0,OPERATIONAL,WholeLifeGo,5.0,152,"340 14 St NW, Calgary",51.054721,-114.094418,51.056071,-114.093224,51.053371,-114.095923,True,NaN,NaN
1,CLOSED_TEMPORARILY,Vivaan's Yummies. Indian Street Food,4.9,156,"3309 19 St NE, Calgary",51.082129,-114.012339,51.083476,-114.010697,51.080777,-114.013397,NaN,True,NaN


In [11]:
#create table in sql
df_sql = df1.to_sql('google_businesses', connection, if_exists='fail', index=True)

ValueError: Table 'google_businesses' already exists.

In [ ]:
#Selecting records
def execute_read_query(connection, query):
    cursor= connection.cursor()
    result = None
    try:
        cursor.execute(query)
        result=cursor.fetchall()
        return result
    except Error as e:
        print(f"The error '{e}' occurred")

In [ ]:
#google = "SELECT * from yelp_businesses"
#google_bus = execute_read_query(connection, google)

#for bus in google_bus:
 #   print(bus)

In [14]:
top_10_POI = df1.sort_values(by=["rating"], ascending= False)
top_10_POI.head(10)

,business_status,name,rating,user_ratings_total,vicinity,geometry.location.lat,geometry.location.lng,geometry.viewport.northeast.lat,geometry.viewport.northeast.lng,geometry.viewport.southwest.lat,geometry.viewport.southwest.lng,opening_hours.open_now,permanently_closed,price_level
0,OPERATIONAL,WholeLifeGo,5.0,152,"340 14 St NW, Calgary",51.054721,-114.094418,51.056071,-114.093224,51.053371,-114.095923,True,NaN,NaN
9,OPERATIONAL,People Food,5.0,1,Calgary,51.048615,-114.070846,51.049985,-114.069495,51.047285,-114.072194,True,NaN,NaN
1,CLOSED_TEMPORARILY,Vivaan's Yummies. Indian Street Food,4.9,156,"3309 19 St NE, Calgary",51.082129,-114.012339,51.083476,-114.010697,51.080777,-114.013397,NaN,True,NaN
7,OPERATIONAL,Damascus Fine Mediterranean Foods,4.8,963,"718 17 Ave SW #4, Calgary",51.038154,-114.077881,51.039340,-114.076632,51.036640,-114.079332,True,NaN,1.0
18,OPERATIONAL,Mondays Plant Café,4.8,290,"208 4 St NE, Calgary",51.053740,-114.051470,51.055090,-114.050120,51.052390,-114.052820,True,NaN,1.0
15,OPERATIONAL,Yenny Delights,4.8,60,"3131 27 St NE #81, Calgary",51.080178,-113.996245,51.081723,-113.994478,51.079024,-113.997178,True,NaN,NaN
13,OPERATIONAL,Better Prep,4.8,40,"1420 40 Ave NE, Calgary",51.089622,-114.029157,51.090515,-114.027811,51.087816,-114.030511,True,NaN,NaN
19,OPERATIONAL,FoodNVibes Caribbean Restaurant & lounge,4.8,110,"2316 27 Ave NE #6a, Calgary",51.077207,-114.006179,51.078422,-114.004526,51.075723,-114.007226,True,NaN,NaN
12,OPERATIONAL,Bridgette Bar,4.7,1736,"739 10 Ave SW, Calgary",51.043668,-114.078671,51.045020,-114.077460,51.042320,-114.080159,True,NaN,3.0
6,OPERATIONAL,Meltwich Food Co.,4.6,501,"1610 10 St SW, Calgary",51.038176,-114.086550,51.039568,-114.085212,51.036868,-114.087912,True,NaN,NaN


In [23]:
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

In [27]:
def create_data_model():
    """Stores the data for the problem."""
    data = {}
    # Locations in block units
    data['locations'] = [
        (51.054721 ,-114.094418),(51.048615, -114.070846), 
        (51.082129 ,-114.012339),(51.038154,-114.077881),
        (51.053740 ,-114.051470),(51.080178 ,-113.996245),
        (51.089622 ,-114.029157), (51.077207 ,-114.006179),
        (51.043668 ,-114.078671),(51.038176,-114.086550 )
        
         ]  # yapf: disable
    data['num_vehicles'] = 1
    data['depot'] = 0
    return data


In [28]:
def compute_euclidean_distance_matrix(locations):
    """Creates callback to return distance between points."""
    distances = {}
    for from_counter, from_node in enumerate(locations):
        distances[from_counter] = {}
        for to_counter, to_node in enumerate(locations):
            if from_counter == to_counter:
                distances[from_counter][to_counter] = 0
            else:
                # Euclidean distance
                distances[from_counter][to_counter] = (int(
                    math.hypot((from_node[0] - to_node[0]),
                               (from_node[1] - to_node[1]))))
    return distances

In [31]:
 data['locations'] = [
        (51.054721 ,-114.094418),(51.048615, -114.070846), 
        (51.082129 ,-114.012339),(51.038154,-114.077881),
        (51.053740 ,-114.051470),(51.080178 ,-113.996245),
        (51.089622 ,-114.029157), (51.077207 ,-114.006179),
        (51.043668 ,-114.078671),(51.038176,-114.086550 )
        
         ]

distance_matrix = compute_euclidean_distance_matrix(data['locations'])

def distance_callback(from_index, to_index):
    """Returns the distance between the two nodes."""
    # Convert from routing variable Index to distance matrix NodeIndex.
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return distance_matrix[from_node][to_node]

transit_callback_index = routing.RegisterTransitCallback(distance_callback)
routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

NameError: name 'data' is not defined